# imports

In [1]:
from warnings import filterwarnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score, roc_curve
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import optuna

sns.set()
filterwarnings("ignore")

# getting and preprocessing data

In [2]:
DATA_DIR = 'kaggle/input'

df_train = pd.read_csv(DATA_DIR +'/train.csv')
df_test = pd.read_csv(DATA_DIR +'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
df_test['default'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [3]:
num_cols = ['age', 'score_bki', 'decline_app_cnt', 'bki_request_cnt', 'income']
cat_cols = ['education', 'first_time', 'sna', 'work_address', 'home_address', 'region_rating']
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

In [4]:
ids = data["client_id"]  # сохраним на всякий пожарный
data.drop(['client_id','app_date',], axis = 1, inplace=True)

# Заполнение пропуски 'education' наиболее частым значением 'SCH'
# Можно было бы выделить в отдельную категорию, "без образования", пробовал, качество от этого не повышается 
data["education"].fillna(data.education.mode(), inplace=True)

# dummies
data = pd.get_dummies(data, columns=['education'], dummy_na=True)

# label encoding
le = LabelEncoder()

for column in bin_cols:
    data[column] = le.fit_transform(data[column])
    
columns = ['first_time', 'sna', 'work_address', 'home_address', 'region_rating']

for column in columns:
    data[column] = le.fit_transform(data[column])

# логорифмируем хвостатых
data['age'] = np.log(data['age'] + 1)
data['decline_app_cnt'] = np.log(data['decline_app_cnt'] + 1)
data['income'] = np.log(data['income'] + 1)
data['bki_request_cnt'] = np.log(data.bki_request_cnt + 1)

# Нормализируем численные
for column in num_cols:
    data[column] = StandardScaler().fit_transform(np.array(data[column].values).reshape(-1, 1))

In [5]:
X, y = (
    data.query("sample == 1").drop(columns=["sample", "default"]),
    data.query("sample == 1")["default"].values,
)
test = data.query("sample == 0").drop(columns=["sample", "default"])

In [6]:
rus = RandomUnderSampler(random_state=42)
X, y = rus.fit_resample(X, y)

# modelling

### model tuning

In [7]:
def objective(trial):
    params = {
        "penalty": trial.suggest_categorical("penalty", ["l1", "l2"]),
        "solver": trial.suggest_categorical("solver", ["liblinear", "lbfgs", "newton-cg", "liblinear", "saga"]),
        "multi_class": trial.suggest_categorical("multi_class", ["auto", "ovr"]),
        "C": trial.suggest_float("C", 1e-10, 1e10, log=True),
    }

    lr = LogisticRegression(**params)

    cv = cross_val_score(lr, X, y, scoring="roc_auc", cv=8, n_jobs=-1)
    return np.mean(cv)


study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///LogisticRegression.db",
    study_name="LogisticRegression",
    load_if_exists=True,
)
study.optimize(objective, timeout=600, n_trials=1)


[I 2022-02-14 22:29:15,624] Using an existing study with name 'LogisticRegression' instead of creating a new one.
[I 2022-02-14 22:29:19,837] Trial 301 finished with value: 0.7255930617045028 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'multi_class': 'ovr', 'C': 0.13090094024167206}. Best is trial 267 with value: 0.7256045378501499.


In [8]:
study.best_params

{'C': 0.07213528501783559,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'solver': 'liblinear'}

In [15]:
def objective(trial):

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = LGBMClassifier(**param, silent=True)
    cv_roc_auc = cross_val_score(gbm, X, y, cv=8, scoring="roc_auc", n_jobs=-1)

    return np.mean(cv_roc_auc)


study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///LGBMClassifier.db",
    study_name="LGBMClassifier",
    load_if_exists=True,
)
study.optimize(objective, timeout=600, n_trials=300)

[I 2022-02-14 22:37:19,595] A new study created in RDB with name: LGBMClassifier
/home/user/opt/mambaforge/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] bagging_fraction is set=0.7782025213312909, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7782025213312909
[LightGBM] [Warning] lambda_l1 is set=0.37758522410982887, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.37758522410982887
[LightGBM] [Warning] feature_fraction is set=0.8659737190263921, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8659737190263921
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] lambda_l2 is set=0.16563585772910563, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.16563585772910563


/home/user/opt/mambaforge/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] bagging_fraction is set=0.7782025213312909, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7782025213312909
[LightGBM] [Warning] lambda_l1 is set=0.37758522410982887, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.37758522410982887
[LightGBM] [Warning] feature_fraction is set=0.8659737190263921, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8659737190263921
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] lambda_l2 is set=0.16563585772910563, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.16563585772910563


/home/user/opt/mambaforge/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


...

[I 2022-02-14 22:47:18,361] Trial 224 finished with value: 0.7267873094959822 and parameters: {'lambda_l1': 0.43371038086506425, 'lambda_l2': 0.003456279447768829, 'num_leaves': 15, 'feature_fraction': 0.41700038457058236, 'bagging_fraction': 0.6029953684949945, 'bagging_freq': 3, 'min_child_samples': 78}. Best is trial 152 with value: 0.7290663262926876.
/home/user/opt/mambaforge/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/home/user/opt/mambaforge/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a 

[LightGBM] [Warning] bagging_fraction is set=0.6397689034927374, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6397689034927374
[LightGBM] [Warning] lambda_l1 is set=0.9111675417385898, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9111675417385898
[LightGBM] [Warning] feature_fraction is set=0.4578706079384927, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4578706079384927
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=0.016324791581261905, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.016324791581261905
[LightGBM] [Warning] bagging_fraction is set=0.6397689034927374, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6397689034927374
[LightGBM] [Warning] lambda_l1 is set=0.9111675417385898, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9111675417385898
[LightGBM] [Warning] feature_fraction is

/home/user/opt/mambaforge/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/home/user/opt/mambaforge/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/home/user/opt/mambaforge/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of Ligh

[LightGBM] [Warning] bagging_fraction is set=0.6397689034927374, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6397689034927374
[LightGBM] [Warning] lambda_l1 is set=0.9111675417385898, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9111675417385898
[LightGBM] [Warning] feature_fraction is set=0.4578706079384927, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4578706079384927
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=0.016324791581261905, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.016324791581261905
[LightGBM] [Warning] bagging_fraction is set=0.6397689034927374, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6397689034927374
[LightGBM] [Warning] lambda_l1 is set=0.9111675417385898, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9111675417385898
[LightGBM] [Warning] feature_fraction is

[I 2022-02-14 22:47:20,283] Trial 225 finished with value: 0.7282191408265156 and parameters: {'lambda_l1': 0.9111675417385898, 'lambda_l2': 0.016324791581261905, 'num_leaves': 6, 'feature_fraction': 0.4578706079384927, 'bagging_fraction': 0.6397689034927374, 'bagging_freq': 3, 'min_child_samples': 85}. Best is trial 152 with value: 0.7290663262926876.


In [16]:
study.best_params

{'bagging_fraction': 0.6641812860565748,
 'bagging_freq': 3,
 'feature_fraction': 0.4257771924417812,
 'lambda_l1': 0.41391873003799096,
 'lambda_l2': 0.009238732952784297,
 'min_child_samples': 76,
 'num_leaves': 9}

### base models

In [17]:
simple_models = [
    LinearRegression(),
    LogisticRegression(
        **{
            "C": 0.07213528501783559,
            "multi_class": "ovr",
            "penalty": "l2",
            "solver": "liblinear",
        },
        random_state=42
    ),
    RandomForestClassifier(n_estimators=256, random_state=42),
    LGBMClassifier(
        **{
            "bagging_fraction": 0.6641812860565748,
            "bagging_freq": 3,
            "feature_fraction": 0.4257771924417812,
            "lambda_l1": 0.41391873003799096,
            "lambda_l2": 0.009238732952784297,
            "min_child_samples": 76,
            "num_leaves": 9,
        },
        random_state=42,
        silent=True
    ),
    CatBoostClassifier(random_state=42, silent=True),
]
simple_models_names = ["linear", "logistic", "rf", "lgbm", "catboost"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, random_state=42, test_size=0.2, shuffle=True
)


In [18]:
meta_df = pd.DataFrame()
for name, model in zip(simple_models_names, simple_models):
    print("Fitting", name, end = " ")
    if name != "linear":
        model.fit(X_train, y_train)
        meta_df[name] = model.predict_proba(X_val)[:,1]
    else:
        model.fit(X_train, y_train)
        meta_df[name] = model.predict(X_val)
    print("done")

meta_df.head(3)

Fitting linear done
Fitting logistic done
Fitting rf done
Fitting lgbm [LightGBM] [Warning] bagging_fraction is set=0.6641812860565748, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6641812860565748
[LightGBM] [Warning] lambda_l1 is set=0.41391873003799096, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.41391873003799096
[LightGBM] [Warning] feature_fraction is set=0.4257771924417812, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4257771924417812
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=0.009238732952784297, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.009238732952784297
done
Fitting catboost done


,linear,logistic,rf,lgbm,catboost
0,0.391481,0.376763,0.328125,0.382711,0.335971
1,0.654895,0.672962,0.707031,0.665023,0.654139
2,0.628189,0.641612,0.667969,0.716308,0.736752


### meta model

In [19]:
meta_model = CatBoostClassifier(random_state=42, silent=True)
meta_model.fit(meta_df, y_val)

test_meta_df = pd.DataFrame()
for name, model in zip(simple_models_names, simple_models):
    print("Predicting using", name, end = " ")
    if name != "linear":
        test_meta_df[name] = model.predict_proba(test)[:,1]
    else:
        test_meta_df[name] = model.predict(test)
    print("done")

test_meta_df["preds"] = meta_model.predict(test_meta_df)
test_meta_df.head(3)

Predicting using linear done
Predicting using logistic done
Predicting using rf done
Predicting using lgbm done
Predicting using catboost done


,linear,logistic,rf,lgbm,catboost,preds
0,0.257588,0.245833,0.207031,0.209238,0.178658,0
1,0.733078,0.754758,0.820312,0.692517,0.927645,1
2,0.336925,0.320274,0.367188,0.358034,0.443330,0


In [20]:
sample_submission[:3]

,client_id,default
0,74835,0
1,17527,1
2,75683,0


In [21]:
sample_submission["default"] = test_meta_df["preds"]

In [22]:
sample_submission.to_csv("submission.csv", index=False)

## kaggle: 0.34142